# functools.wrapsを使って関数デコレータを定義する

In [5]:
from functools import *

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
             (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci = trace(fibonacci)

fibonacci(3)

print(fibonacci)
help(fibonacci)

fibonacci((1,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((3,), {}) -> 2
<function fibonacci at 0x7fe1ec532c80>
Help on function fibonacci in module __main__:

fibonacci(n)



# contextlibとwith文をtry/finallyの代わりに考える

In [16]:
import logging
from contextlib import *

def my_function():
    logging.debug("debug")
    logging.error("Error debug")
    logging.debug("More debug")
    


@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)
    
with debug_logging(logging.DEBUG):
    print("Inside")
    my_function()
print("After")
my_function()

@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

with log_level(logging.DEBUG, 'my-log') as logger:
    logging.debug("print debug")
    logging.debug("print Error debug")

DEBUG:root:debug
ERROR:root:Error debug
DEBUG:root:More debug
ERROR:root:Error debug


Inside
After


# copyregでpickleを信頼できるようにする

In [32]:
import copyreg
import pickle

class GameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic
        
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1:
        kwargs.pop('lives')
    return GameState(**kwargs)

copyreg.pickle(GameState, pickle_game_state)

state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)


class BetterGameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic
      
state = BetterGameState()
serialized = pickle.dumps(state)
print(serialized[:35])

{'magic': 5, 'points': 1000, 'level': 0}
b'\x80\x03c__main__\nBetterGameState\nq\x00)\x81q\x01}'


# ローカル時間には`datetime`を使用

In [38]:
from datetime import *
import pytz

time_format = '%Y-%m-%d %H:%M:%S'

arrival_nyc = '2014-05-01 23:33:24'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')
nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2014-05-02 03:33:24+00:00


# 組み込みアルゴリズムとデータ構造

In [51]:

from collections import *
from heapq import *
from bisect import *

# double-ended Queue

fifo = deque()
fifo.append(1)
x = fifo.popleft()

# Ordered Dict

a = OrderedDict()
a['foo'] = 1
a['bar'] = 2

b = OrderedDict()
b['foo'] = 'red'
b['bar'] = 'blue'

for value1, value2 in zip(a.values(), b.values()):
    print(value1, value2)
    
# default Dict

stats = defaultdict(int)
stats['my_counter'] += 1
print(stats)

#heap queue

a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)

print(heappop(a), heappop(a), heappop(a), heappop(a))

a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)


print('Before ', a)
a.sort()
print('After ',a)

# bisection

x = list(range(10**6))
i = x.index(991234)

i - bisect_left(x, 991234)

1 red
2 blue
defaultdict(<class 'int'>, {'my_counter': 1})
3 4 5 7
Before  [3, 4, 7, 5]
After  [3, 4, 5, 7]


0

# 正確な計算にはDecimalを使用する

In [54]:
from decimal import Decimal


rate = Decimal('1.45')
seconds = Decimal('222')
cost = rate * seconds / Decimal('60')
print(cost)

5.365
